# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import datasets
import evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig, EarlyStoppingCallback
from transformers.trainer_utils import get_last_checkpoint
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from accelerate import cpu_offload
import sqlite3
import sqlparse
from tqdm.auto import tqdm
from datetime import datetime
import pickle
import wandb
import psutil
import GPUtil
import os
import gc
import math

import _config

In [2]:
ENABLE_THINKING = False

In [3]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

os.environ["WANDB_API_KEY"] = _config.WANDB_API_KEY
os.environ["WANDB_PROJECT"] = _config.WANDB_PROJECT

# Utils

In [4]:
def get_vm_usage_metrics():
    # CPU usage
    cpu_load = psutil.cpu_percent(interval=1, percpu=True)
    for id, load in enumerate(cpu_load):
        print(f"CPU {id} load: {load:.2f}")
    # RAM usage
    ram = psutil.virtual_memory()
    print(f"RAM Total: {ram.total/(1024**3):.2f} GB, Used: {(ram.used)/(1024**3):.2f} GB")
    # GPU
    if torch.cuda.is_available():
        gpus = GPUtil.getGPUs()
        for gpu in gpus:
            print(f"GPU {gpu.id} ({gpu.name}) load: {gpu.load*100}%")
            print(f"GPU {gpu.id} ({gpu.name}) VRAM Total: {gpu.memoryTotal} MB, Used {gpu.memoryUsed} MB")
    # Disk 
    disk = psutil.disk_usage('/')
    print(f"Disk Total: {disk.total/(1024**3):.2f} GB, Used: {(disk.used)/(1024**3):.2f} GB")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f'Device: {device}')
get_vm_usage_metrics()

Device: cuda
CPU 0 load: 0.00
CPU 1 load: 0.00
CPU 2 load: 2.00
CPU 3 load: 0.00
RAM Total: 27.41 GB, Used: 1.62 GB
GPU 0 (Tesla T4) load: 0.0%
GPU 0 (Tesla T4) VRAM Total: 16384.0 MB, Used 3.0 MB
Disk Total: 60.95 GB, Used: 38.36 GB


In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"Trainable params: {trainable_params} || All params: {all_param} || Trainable %: {100 * trainable_params / all_param:.2f}"
    )

# Data

In [6]:
ds = datasets.load_dataset('gretelai/synthetic_text_to_sql', streaming=False)
ds_train, ds_test = ds['train'], ds['test']

split = ds_train.train_test_split(test_size=0.025, seed=42)
ds_train = split['train']
ds_valid = split['test']

ds_train

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
    num_rows: 97500
})

# Model

In [ ]:
checkpoint = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    device_map="cuda",
)
# model = cpu_offload(model)

print_trainable_parameters(model)
get_vm_usage_metrics()

In [7]:
def construct_message(prompt, context):
    return [
        {"role": "system", "content": f"The user asks a question. Your task is to generate the SQL query to answer that question. Return SQL query only. The context of the question is the following: '{context}'"},
        {"role": "user", "content": prompt}
    ]

In [8]:
def generate_model_response_batch(model, tokenizer, messages_list, enable_thinking=True, max_new_tokens=512):
    texts = [
        tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True,
            enable_thinking=enable_thinking
        )
        for messages in messages_list
    ]

    model_inputs = tokenizer(
        texts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        padding_side='left'
    ).to(model.device)

    model.eval()
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=max_new_tokens
    )

    responses = []
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids):
        # Slice to get only generated part
        output_only_ids = output_ids[len(input_ids):].tolist()

        # Try to find `</think>` (id 151668)
        try:
            index = len(output_only_ids) - output_only_ids[::-1].index(151668)
        except ValueError:
            index = 0

        if enable_thinking:
            thinking_content = tokenizer.decode(
                output_only_ids[:index],
                skip_special_tokens=True
            ).strip("\n")
            content = tokenizer.decode(
                output_only_ids[index:],
                skip_special_tokens=True
            ).strip("\n")
        else:
            thinking_content = None
            content = tokenizer.decode(
                output_only_ids,
                skip_special_tokens=True
            ).strip("\n")

        responses.append({
            'thinking_content': thinking_content,
            'content': content
        })

    return responses

# Evaluate

In [9]:
rouge = evaluate.load("rouge")

def normalize_sql(sql):
    return sqlparse.format(sql, reindent=True, keyword_case='upper').strip()

def compute_rouge(reference, prediction):
    result = rouge.compute(predictions=[prediction], references=[reference])
    return result['rougeL']

def evaluate_sql_response(reference, prediction, sql_context):
    # ROUGE-L
    rouge_score = compute_rouge(reference, prediction)
    
    # execution check
    try:
        conn = sqlite3.connect(":memory:")
        cursor = conn.cursor()
        
        cursor.executescript(sql_context)
        cursor.execute(reference)
        ref_result = cursor.fetchall()
        
        cursor.execute(prediction)
        model_result = cursor.fetchall()
        
        execution_match = ref_result == model_result
    except Exception:
        execution_match = False
    finally:
        conn.close()
    
    # final score
    if execution_match:
        final_score = 1.0
    else:
        final_score = 0.7 * rouge_score

    return {
        "rougeL": round(rouge_score, 4),
        "execution_match": execution_match,
        "final_score": final_score
    }

# Formatting functions

In [10]:
# used for training
def construct_message_with_assistant_content(example):
    messages = construct_message(example['sql_prompt'], example['sql_context'])
    messages.append({
        'role': 'assistant',
        'content': example['sql']
    })
    return {'messages': messages}

In [11]:
def formatting_func(example, enable_thinking=ENABLE_THINKING):
    return tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False, # no generation prompt during training
        enable_thinking=ENABLE_THINKING 
    )

# LoRA- step 1

In [12]:
TRAIN_SIZE = 4096
VALID_SIZE = 1024

ds_train_sample = ds_train.take(TRAIN_SIZE)
ds_valid_sample = ds_valid.take(VALID_SIZE)

len(ds_train_sample), len(ds_valid_sample), len(ds_test)

ds_train_with_assistant_content = ds_train_sample.map(construct_message_with_assistant_content)
ds_valid_with_assistant_content = ds_valid_sample.map(construct_message_with_assistant_content)

In [13]:
get_vm_usage_metrics()

CPU 0 load: 1.00
CPU 1 load: 0.00
CPU 2 load: 1.00
CPU 3 load: 0.00
RAM Total: 27.41 GB, Used: 1.79 GB
GPU 0 (Tesla T4) load: 0.0%
GPU 0 (Tesla T4) VRAM Total: 16384.0 MB, Used 2987.0 MB
Disk Total: 60.95 GB, Used: 32.28 GB


In [14]:
torch.cuda.empty_cache()

In [ ]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

sweep_config = {
    'name': f'sweep-lora-step1-epochs1-samples{TRAIN_SIZE}-{timestamp}',
    'method': 'bayes',
    'metric': {
        'name': 'eval_loss',
        'goal': 'minimize'   
    },
    'parameters': {
        'optimizer': {'values': ['adam', 'adamw', 'nadam', 'adamax']},
        'effective_batch_size': {'values': [16, 32, 64, 128, 256, 512]},
        'learning_rate': {'values': [1e-4, 5e-5, 1e-5, 5e-6, 1e-6]},
        'weight_decay': {'values': [0.0, 0.01, 0.1]},
        'betas': {'values': [(0.9, 0.999), (0.95, 0.999), (0.9, 0.9999)]},
        'warmup_ratio': {'values': [0.05, 0.1, 0.2]},
        'epochs': {'values': [1]},
        'lora_r': {'values': [4, 8, 16, 32]},
        'lora_alpha': {'values': [2, 4, 8, 16, 32, 64]},
        'lora_dropout': {'values': [0.01, 0.05, 0.1, 0.2]}
    }
}

# sweep_id = wandb.sweep(sweep_config)
sweep_id = '9a4oj3so' # continue the crashed sweep

optimizer_map = {
    "adam": torch.optim.Adam,
    "adamw": torch.optim.AdamW,
    "nadam": torch.optim.NAdam,
    "adamax": torch.optim.Adamax
}

def sweep_train():
    with wandb.init() as run:
        model = AutoModelForCausalLM.from_pretrained(checkpoint)
        config = wandb.config  
        PER_DEVICE_BATCH_SIZE = 2  # higher values --> OOM
        gradient_accumulation_steps = int(config.effective_batch_size / PER_DEVICE_BATCH_SIZE)
        
        training_args = TrainingArguments(
            per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
            gradient_accumulation_steps=gradient_accumulation_steps,
            learning_rate=config.learning_rate,
            num_train_epochs=config.epochs,
            weight_decay=config.weight_decay,
            lr_scheduler_type="cosine",
            warmup_ratio=config.warmup_ratio,
            save_strategy="no",
            eval_strategy="epoch",
            logging_strategy="steps",
            logging_steps=1,
            report_to=['wandb'],
            fp16=True,
            fp16_full_eval=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            max_grad_norm=1,
            # load_best_model_at_end=True
        )
        
        def build_optimizer(model):
            optimizer_class = optimizer_map[config.optimizer]
            return optimizer_class(
                model.parameters(),
                lr=config.learning_rate,
                weight_decay=config.weight_decay,
                betas=config.betas
            )


        peft_config = LoraConfig(
            r=config.lora_r,
            lora_alpha=config.lora_alpha,
            lora_dropout=config.lora_dropout,
            bias="none",
            task_type="CAUSAL_LM"
        )
        model.requires_grad_(False)                     # freeze base weights (precautionary)
        model_peft = get_peft_model(model, peft_config) # inject a LoRA adapter
        
        trainer = SFTTrainer(
            model=model_peft,
            train_dataset=ds_train_with_assistant_content,
            eval_dataset=ds_valid_with_assistant_content,
            formatting_func=formatting_func,
            args=training_args,
            optimizers=(build_optimizer(model_peft), None),  # (optimizer, scheduler)
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
        )

        trainer.train()

        for log in trainer.state.log_history:
            if 'eval_loss' in log:
                wandb.log({
                    "eval_loss": log['eval_loss'],
                    "eval_perplexity": math.exp(log['eval_loss']),
                    "step": log['step'],
                    "learning_rate": config.learning_rate,
                    "weight_decay": config.weight_decay,
                    "betas": config.betas,
                    "warmup_ratio": config.warmup_ratio,
                    "effective_batch_size": config.effective_batch_size,
                    "optimizer": config.optimizer
                })
        wandb.finish(); # finish the run
        del trainer
        torch.cuda.empty_cache()
        gc.collect()
        
wandb.agent(sweep_id, function=sweep_train, count=60)

wandb: Agent Starting Run: a8dgyigc with config:
wandb: 	betas: [0.9, 0.9999]
wandb: 	effective_batch_size: 16
wandb: 	epochs: 1
wandb: 	learning_rate: 0.0001
wandb: 	lora_alpha: 16
wandb: 	lora_dropout: 0.2
wandb: 	lora_r: 8
wandb: 	optimizer: adamw
wandb: 	warmup_ratio: 0.2
wandb: 	weight_decay: 0.1
wandb: Currently logged in as: olialeshka (olialeshka-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_ratio' was locked by 'sweep' (ignored update).
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss


# LoRA - step 2

In [15]:
torch.cuda.empty_cache()
get_vm_usage_metrics()


TRAIN_SIZE = 8192
VALID_SIZE = 2048

ds_train_sample = ds_train.take(TRAIN_SIZE)
ds_valid_sample = ds_valid.take(VALID_SIZE)

print(len(ds_train_sample), len(ds_valid_sample), len(ds_test))

ds_train_with_assistant_content = ds_train_sample.map(construct_message_with_assistant_content)
ds_valid_with_assistant_content = ds_valid_sample.map(construct_message_with_assistant_content)

CPU 0 load: 0.00
CPU 1 load: 0.00
CPU 2 load: 1.00
CPU 3 load: 0.00
RAM Total: 27.41 GB, Used: 1.73 GB
GPU 0 (Tesla T4) load: 0.0%
GPU 0 (Tesla T4) VRAM Total: 16384.0 MB, Used 2407.0 MB
Disk Total: 60.95 GB, Used: 35.50 GB
8192 2048 5851


Map:   0%|          | 0/8192 [00:00<?, ? examples/s]

Map:   0%|          | 0/2048 [00:00<?, ? examples/s]

In [ ]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

sweep_config = {
    'name': f'sweep-lora-step2-epochs1-samples{TRAIN_SIZE}-{timestamp}',
    'method': 'grid',
    'metric': {
        'name': 'eval_loss',
        'goal': 'minimize'   
    },
    'parameters': {
        'optimizer': {'values': ['adamw', 'nadam']},
        'effective_batch_size': {'values': [16]},
        'learning_rate': {'values': [1e-5]}, # best results from bayes search have 1e-4 -> setting to a lower value
        'weight_decay': {'values': [0.0]},
        'betas': {'values': [(0.9, 0.9999)]},
        'warmup_ratio': {'values': [0.05, 0.1, 0.2]},
        'epochs': {'values': [1]},
        'lora_r': {'values': [8, 16, 32]},
        'lora_alpha': {'values': [64]},
        'lora_dropout': {'values': [0.01, 0.05, 0.1, 0.2]}
    }
}

sweep_id = wandb.sweep(sweep_config)
# sweep_id = '9a4oj3so' # continue the crashed sweep

optimizer_map = {
    "adam": torch.optim.Adam,
    "adamw": torch.optim.AdamW,
    "nadam": torch.optim.NAdam,
    "adamax": torch.optim.Adamax
}

def sweep_train():
    with wandb.init() as run:
        model = AutoModelForCausalLM.from_pretrained(checkpoint)
        config = wandb.config  
        PER_DEVICE_BATCH_SIZE = 2  # higher values --> OOM
        gradient_accumulation_steps = int(config.effective_batch_size / PER_DEVICE_BATCH_SIZE)
        
        training_args = TrainingArguments(
            per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
            gradient_accumulation_steps=gradient_accumulation_steps,
            learning_rate=config.learning_rate,
            num_train_epochs=config.epochs,
            weight_decay=config.weight_decay,
            lr_scheduler_type="cosine",
            warmup_ratio=config.warmup_ratio,
            save_strategy="no",
            eval_strategy="epoch",
            logging_strategy="steps",
            logging_steps=1,
            report_to=['wandb'],
            fp16=True,
            fp16_full_eval=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            max_grad_norm=1,
            # load_best_model_at_end=True
        )
        
        def build_optimizer(model):
            optimizer_class = optimizer_map[config.optimizer]
            return optimizer_class(
                model.parameters(),
                lr=config.learning_rate,
                weight_decay=config.weight_decay,
                betas=config.betas
            )


        peft_config = LoraConfig(
            r=config.lora_r,
            lora_alpha=config.lora_alpha,
            lora_dropout=config.lora_dropout,
            bias="none",
            task_type="CAUSAL_LM"
        )
        model.requires_grad_(False)                     # freeze base weights (precautionary)
        model_peft = get_peft_model(model, peft_config) # inject a LoRA adapter
        
        trainer = SFTTrainer(
            model=model_peft,
            train_dataset=ds_train_with_assistant_content,
            eval_dataset=ds_valid_with_assistant_content,
            formatting_func=formatting_func,
            args=training_args,
            optimizers=(build_optimizer(model_peft), None),  # (optimizer, scheduler)
            callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
        )

        trainer.train()

        for log in trainer.state.log_history:
            if 'eval_loss' in log:
                wandb.log({
                    "eval_loss": log['eval_loss'],
                    "eval_perplexity": math.exp(log['eval_loss']),
                    "step": log['step'],
                    "learning_rate": config.learning_rate,
                    "weight_decay": config.weight_decay,
                    "betas": config.betas,
                    "warmup_ratio": config.warmup_ratio,
                    "effective_batch_size": config.effective_batch_size,
                    "optimizer": config.optimizer
                })
        wandb.finish(); # finish the run
        del trainer
        torch.cuda.empty_cache()
        gc.collect()
        
wandb.agent(sweep_id, function=sweep_train)

Create sweep with ID: ub224w5u
Sweep URL: https://wandb.ai/olialeshka-none/text-to-sql/sweeps/ub224w5u


wandb: Agent Starting Run: d1jy1p86 with config:
wandb: 	betas: [0.9, 0.9999]
wandb: 	effective_batch_size: 16
wandb: 	epochs: 1
wandb: 	learning_rate: 1e-05
wandb: 	lora_alpha: 64
wandb: 	lora_dropout: 0.01
wandb: 	lora_r: 8
wandb: 	optimizer: adamw
wandb: 	warmup_ratio: 0.05
wandb: 	weight_decay: 0


wandb: 
wandb: 🚀 View run floral-sweep-1 at: https://wandb.ai/olialeshka-none/text-to-sql/runs/r450607u
wandb: Find logs at: wandb/run-20251004_103745-r450607u/logs


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_ratio' was locked by 'sweep' (ignored update).
Using EarlyStoppingCallback without load_best_model_at_end=True. Once training is finished, the best model will not be loaded automatically.


Epoch,Training Loss,Validation Loss


# LoRA - step 3 - the final model

In [12]:
torch.cuda.empty_cache()
get_vm_usage_metrics()

print(len(ds_train), len(ds_valid), len(ds_test))

ds_train_with_assistant_content = ds_train.map(construct_message_with_assistant_content)
ds_valid_with_assistant_content = ds_valid.map(construct_message_with_assistant_content)

CPU 0 load: 0.00
CPU 1 load: 0.00
CPU 2 load: 0.00
CPU 3 load: 0.00
RAM Total: 27.41 GB, Used: 1.84 GB
GPU 0 (Tesla T4) load: 0.0%
GPU 0 (Tesla T4) VRAM Total: 16384.0 MB, Used 2407.0 MB
Disk Total: 60.95 GB, Used: 32.44 GB
97500 2500 5851


In [ ]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# resuming the prev run
# timestamp = '2025-08-19_08-33-30'
RUN_NAME = f'lora-final-model-{timestamp}'
# run_id = 'imoh6jtd'
wandb.init(
    project=os.environ["WANDB_PROJECT"],
    name=RUN_NAME,
    # id=run_id,         # resume previous run if available
    resume="allow",    # allows resuming crashed run
)


RESUME_TRAINING = True
OUTPUT_DIR = "./lora-final_model-output"
PER_DEVICE_BATCH_SIZE = 2  # higher values --> OOM

optimizer = 'nadam'
effective_batch_size = 16
learning_rate = 1e-5
weight_decay = 0.0
betas = (0.9, 0.9999)
warmup_ratio = 0.2
epochs = 1
gradient_accumulation_steps = int(effective_batch_size / PER_DEVICE_BATCH_SIZE)
lora_r = 16
lora_alpha = 64
lora_dropout = 0.01

optimizer_map = {
    "adam": torch.optim.Adam,
    "adamw": torch.optim.AdamW,
    "nadam": torch.optim.NAdam,
    "adamax": torch.optim.Adamax
}

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    lr_scheduler_type="cosine",
    warmup_ratio=warmup_ratio,
    save_strategy="steps",
    save_steps=gradient_accumulation_steps*5,
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=gradient_accumulation_steps*5,
    logging_strategy="steps",
    logging_steps=gradient_accumulation_steps*5,
    report_to=['wandb'],
    run_name=RUN_NAME,
    fp16=True,
    fp16_full_eval=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    max_grad_norm=1,
    load_best_model_at_end=True,
    gradient_checkpointing=True
)

def build_optimizer(model):
    optimizer_class = optimizer_map[optimizer]
    return optimizer_class(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay,
        betas=betas
    )


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM"
)
model.requires_grad_(False)                     # freeze base weights (precautionary)
model_peft = get_peft_model(model, peft_config) # inject a LoRA adapter

trainer = SFTTrainer(
    model=model_peft,
    train_dataset=ds_train_with_assistant_content,
    eval_dataset=ds_valid_with_assistant_content,
    formatting_func=formatting_func,
    args=training_args,
    optimizers=(build_optimizer(model_peft), None),  # (optimizer, scheduler)
    callbacks=[EarlyStoppingCallback(early_stopping_patience=25)]
)


# Training setup summary
dataset_size = len(ds_train_with_assistant_content)
steps_per_epoch = dataset_size // (PER_DEVICE_BATCH_SIZE * gradient_accumulation_steps)
total_steps = steps_per_epoch * epochs
warmup_steps = int(total_steps * warmup_ratio)

print("===== Training Setup Summary =====")
print(f"Num epochs:            {epochs}")
print(f"Effective batch size:  {effective_batch_size}")
print(f"Per-device batch size: {PER_DEVICE_BATCH_SIZE}")
print(f"Gradient accumulation: {gradient_accumulation_steps}")
print(f"Dataset size:          {dataset_size}")
print(f"Steps per epoch:       {steps_per_epoch}")
print(f"Total training steps:  {total_steps}")
print(f"Warmup steps:          {warmup_steps}")
print(f"Logging steps:         {training_args.logging_steps}")
print("===================================")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")


# Training
last_checkpoint = None
if RESUME_TRAINING and os.path.isdir(OUTPUT_DIR):
    last_checkpoint = get_last_checkpoint(OUTPUT_DIR)

if last_checkpoint is not None:
    print(f"Resuming training from checkpoint: {last_checkpoint}")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("Starting fresh training run")
    trainer.train()

print(f"End time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")


# WandB logging of eval metrics
for log in trainer.state.log_history:
    if 'eval_loss' in log:
        wandb.log({
            "eval_loss": log['eval_loss'],
            "eval_perplexity": math.exp(log['eval_loss']),
            "step": log['step'],
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "betas": betas,
            "warmup_ratio": warmup_ratio,
            "effective_batch_size": effective_batch_size,
            "optimizer": optimizer
        })

wandb.finish()  # finish the run

wandb: Currently logged in as: olialeshka (olialeshka-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Applying formatting function to train dataset:   0%|          | 0/97500 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/97500 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/97500 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/2500 [00:00<?, ? examples/s]

...

Truncating eval dataset:   0%|          | 0/2500 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


===== Training Setup Summary =====
Num epochs:            1
Effective batch size:  16
Per-device batch size: 2
Gradient accumulation: 8
Dataset size:          97500
Steps per epoch:       6093
Total training steps:  6093
Warmup steps:          1218
Logging steps:         40
Start time: 2025-10-05_06-32-05
Starting fresh training run


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
40,2.211400,2.221138,1.160082,124369.000000,0.661763
80,2.188700,2.151783,1.170986,246617.000000,0.666132
120,2.063300,2.045051,1.174300,372357.000000,0.674203
160,1.960600,1.898500,1.163218,496224.000000,0.690094
200,1.763900,1.712549,1.134681,624347.000000,0.711704
240,1.575200,1.465905,1.063029,749722.000000,0.730832
280,1.279200,1.131720,0.885653,875852.000000,0.776071
320,0.986700,0.885480,0.773703,999432.000000,0.808772
360,0.822400,0.773341,0.707053,1123822.000000,0.836914
400,0.735400,0.721101,0.662706,1249546.000000,0.845503


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
model_path = os.path.join(OUTPUT_DIR, 'final')
trainer.save_model(model_path)

## Test

In [15]:
torch.cuda.empty_cache()
get_vm_usage_metrics()

CPU 0 load: 2.00
CPU 1 load: 0.00
CPU 2 load: 1.00
CPU 3 load: 0.00
RAM Total: 27.41 GB, Used: 3.82 GB
GPU 0 (Tesla T4) load: 0.0%
GPU 0 (Tesla T4) VRAM Total: 16384.0 MB, Used 2515.0 MB
Disk Total: 60.95 GB, Used: 33.67 GB


In [16]:
model_path = './lora-final_model-output/final'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
model.eval()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=1024, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.01, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=1024, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): lora.Linear(
            (base_layer): Linear(in_features=1024, out_features=1024

In [ ]:
BATCH_SIZE = 32
ENABLE_THINKING = False
MAX_NEW_TOKENS = 512


prompts = [ds_test[id]['sql_prompt'] for id in range(len(ds_test))]
contexts = [ds_test[id]['sql_context'] for id in range(len(ds_test))]

responses = []
print(f"Start time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")
for i in tqdm(range(0, len(prompts), BATCH_SIZE)):
    batch_prompts = prompts[i : i + BATCH_SIZE]
    batch_contexts = contexts[i : i + BATCH_SIZE]

    messages_list = [
        construct_message(prompt=p, context=c)
        for p, c in zip(batch_prompts, batch_contexts)
    ]

    batch_responses = generate_model_response_batch(model, tokenizer, messages_list, enable_thinking=ENABLE_THINKING, max_new_tokens=MAX_NEW_TOKENS)

    responses.extend(batch_responses)

print(f"End time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")

Start time: 2025-10-05_15-15-10


  0%|          | 0/183 [00:00<?, ?it/s]

In [ ]:
references = [ds_test[id]['sql'] for id in range(len(ds_test))]
predictions = [responses[id]['content'] for id in range(len(ds_test))]

scores = [
    evaluate_sql_response(
        reference=reference,
        prediction=prediction,
        sql_context=context
    )
    for reference, prediction, context in tqdm(zip(references, predictions, contexts), total=len(ds_test))
]

In [21]:
print(f"Mean test set score: {np.mean([score['final_score'] for score in scores]):.3f}")

Mean test set score: 0.736


In [ ]:
with open('lora_test_predictions.pkl', 'wb') as f:
    pickle.dump(predictions, f)

# LoRA - step 4 - the final model (full LoRA)

In [12]:
torch.cuda.empty_cache()
get_vm_usage_metrics()

print(len(ds_train), len(ds_valid), len(ds_test))

ds_train_with_assistant_content = ds_train.map(construct_message_with_assistant_content)
ds_valid_with_assistant_content = ds_valid.map(construct_message_with_assistant_content)

CPU 0 load: 0.00
CPU 1 load: 0.00
CPU 2 load: 0.00
CPU 3 load: 0.00
RAM Total: 27.41 GB, Used: 1.78 GB
GPU 0 (Tesla T4) load: 0.0%
GPU 0 (Tesla T4) VRAM Total: 16384.0 MB, Used 2407.0 MB
Disk Total: 60.95 GB, Used: 33.68 GB
97500 2500 5851


In [ ]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# resuming the prev run
# timestamp = '2025-08-19_08-33-30'
RUN_NAME = f'lora-final-model-all-linear-{timestamp}'
# run_id = 'imoh6jtd'
wandb.init(
    project=os.environ["WANDB_PROJECT"],
    name=RUN_NAME,
    # id=run_id,         # resume previous run if available
    resume="allow",    # allows resuming crashed run
)


RESUME_TRAINING = True
OUTPUT_DIR = "./lora-final_model_all_linear-output"
PER_DEVICE_BATCH_SIZE = 2  # higher values --> OOM

optimizer = 'nadam'
effective_batch_size = 16
learning_rate = 1e-5
weight_decay = 0.0
betas = (0.9, 0.9999)
warmup_ratio = 0.2
epochs = 1
gradient_accumulation_steps = int(effective_batch_size / PER_DEVICE_BATCH_SIZE)
lora_r = 16
lora_alpha = 64
lora_dropout = 0.01

optimizer_map = {
    "adam": torch.optim.Adam,
    "adamw": torch.optim.AdamW,
    "nadam": torch.optim.NAdam,
    "adamax": torch.optim.Adamax
}

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    lr_scheduler_type="cosine",
    warmup_ratio=warmup_ratio,
    save_strategy="steps",
    save_steps=gradient_accumulation_steps*5,
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=gradient_accumulation_steps*5,
    logging_strategy="steps",
    logging_steps=gradient_accumulation_steps*5,
    report_to=['wandb'],
    run_name=RUN_NAME,
    fp16=True,
    fp16_full_eval=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    max_grad_norm=1,
    load_best_model_at_end=True,
    gradient_checkpointing=True
)

def build_optimizer(model):
    optimizer_class = optimizer_map[optimizer]
    return optimizer_class(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay,
        betas=betas
    )


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules='all-linear'
)
model.requires_grad_(False)                     # freeze base weights (precautionary)
model_peft = get_peft_model(model, peft_config) # inject a LoRA adapter

trainer = SFTTrainer(
    model=model_peft,
    train_dataset=ds_train_with_assistant_content,
    eval_dataset=ds_valid_with_assistant_content,
    formatting_func=formatting_func,
    args=training_args,
    optimizers=(build_optimizer(model_peft), None),  # (optimizer, scheduler)
    callbacks=[EarlyStoppingCallback(early_stopping_patience=25)]
)


# Training setup summary
dataset_size = len(ds_train_with_assistant_content)
steps_per_epoch = dataset_size // (PER_DEVICE_BATCH_SIZE * gradient_accumulation_steps)
total_steps = steps_per_epoch * epochs
warmup_steps = int(total_steps * warmup_ratio)

print("===== Training Setup Summary =====")
print(f"Num epochs:            {epochs}")
print(f"Effective batch size:  {effective_batch_size}")
print(f"Per-device batch size: {PER_DEVICE_BATCH_SIZE}")
print(f"Gradient accumulation: {gradient_accumulation_steps}")
print(f"Dataset size:          {dataset_size}")
print(f"Steps per epoch:       {steps_per_epoch}")
print(f"Total training steps:  {total_steps}")
print(f"Warmup steps:          {warmup_steps}")
print(f"Logging steps:         {training_args.logging_steps}")
print("===================================")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")


# Training
last_checkpoint = None
if RESUME_TRAINING and os.path.isdir(OUTPUT_DIR):
    last_checkpoint = get_last_checkpoint(OUTPUT_DIR)

if last_checkpoint is not None:
    print(f"Resuming training from checkpoint: {last_checkpoint}")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("Starting fresh training run")
    trainer.train()

print(f"End time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")


# WandB logging of eval metrics
for log in trainer.state.log_history:
    if 'eval_loss' in log:
        wandb.log({
            "eval_loss": log['eval_loss'],
            "eval_perplexity": math.exp(log['eval_loss']),
            "step": log['step'],
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "betas": betas,
            "warmup_ratio": warmup_ratio,
            "effective_batch_size": effective_batch_size,
            "optimizer": optimizer
        })

wandb.finish()  # finish the run

/home/olialeshka_1/myenv/lib/python3.12/site-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/home/olialeshka_1/myenv/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


===== Training Setup Summary =====
Num epochs:            1
Effective batch size:  16
Per-device batch size: 2
Gradient accumulation: 8
Dataset size:          97500
Steps per epoch:       6093
Total training steps:  6093
Warmup steps:          1218
Logging steps:         40
Start time: 2025-10-06_06-59-49
Starting fresh training run


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
40,2.206400,2.196743,1.164162,124369.000000,0.664029
80,2.106300,1.990358,1.163912,246617.000000,0.681711
120,1.812600,1.678701,1.103828,372357.000000,0.717856
160,1.452700,1.201856,0.853513,496224.000000,0.779109
200,0.981900,0.827730,0.746682,624347.000000,0.818777
240,0.754700,0.714610,0.652080,749722.000000,0.845053
280,0.679500,0.675604,0.634108,875852.000000,0.849026
320,0.651000,0.650463,0.627987,999432.000000,0.851038
360,0.637900,0.627056,0.621941,1123822.000000,0.853274
400,0.608400,0.603315,0.621214,1249546.000000,0.859687


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [17]:
model_path = os.path.join(OUTPUT_DIR, 'final')
trainer.save_model(model_path)

## Test

In [18]:
torch.cuda.empty_cache()
get_vm_usage_metrics()

CPU 0 load: 2.00
CPU 1 load: 1.00
CPU 2 load: 2.00
CPU 3 load: 0.00
RAM Total: 27.41 GB, Used: 2.87 GB
GPU 0 (Tesla T4) load: 0.0%
GPU 0 (Tesla T4) VRAM Total: 16384.0 MB, Used 2647.0 MB
Disk Total: 60.95 GB, Used: 33.99 GB


In [19]:
model_path = './lora-final_model_all_linear-output/final'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)
model.eval()

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=1024, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.01, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=1024, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default

In [ ]:
BATCH_SIZE = 32
ENABLE_THINKING = False
MAX_NEW_TOKENS = 512


prompts = [ds_test[id]['sql_prompt'] for id in range(len(ds_test))]
contexts = [ds_test[id]['sql_context'] for id in range(len(ds_test))]

responses = []
print(f"Start time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")
for i in tqdm(range(0, len(prompts), BATCH_SIZE)):
    batch_prompts = prompts[i : i + BATCH_SIZE]
    batch_contexts = contexts[i : i + BATCH_SIZE]

    messages_list = [
        construct_message(prompt=p, context=c)
        for p, c in zip(batch_prompts, batch_contexts)
    ]

    batch_responses = generate_model_response_batch(model, tokenizer, messages_list, enable_thinking=ENABLE_THINKING, max_new_tokens=MAX_NEW_TOKENS)

    responses.extend(batch_responses)

print(f"End time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")

Start time: 2025-10-06_18-55-47


  0%|          | 0/183 [00:00<?, ?it/s]

In [ ]:
references = [ds_test[id]['sql'] for id in range(len(ds_test))]
predictions = [responses[id]['content'] for id in range(len(ds_test))]

scores = [
    evaluate_sql_response(
        reference=reference,
        prediction=prediction,
        sql_context=context
    )
    for reference, prediction, context in tqdm(zip(references, predictions, contexts), total=len(ds_test))
]

In [23]:
print(f"Mean test set score: {np.mean([score['final_score'] for score in scores]):.3f}")

Mean test set score: 0.756


# LoRA - step 5 - higher rank

In [16]:
torch.cuda.empty_cache()
get_vm_usage_metrics()

print(len(ds_train), len(ds_valid), len(ds_test))

ds_train_with_assistant_content = ds_train.map(construct_message_with_assistant_content)
ds_valid_with_assistant_content = ds_valid.map(construct_message_with_assistant_content)

CPU 0 load: 1.00
CPU 1 load: 2.00
CPU 2 load: 0.00
CPU 3 load: 1.00
RAM Total: 27.41 GB, Used: 1.90 GB
GPU 0 (Tesla T4) load: 0.0%
GPU 0 (Tesla T4) VRAM Total: 16384.0 MB, Used 2407.0 MB
Disk Total: 60.95 GB, Used: 35.11 GB
97500 2500 5851


In [ ]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

# resuming the prev run
timestamp = '2025-10-07_08-21-41'
RUN_NAME = f'lora-final-model-all-linear-r64-{timestamp}'
run_id = 'sla5zrpx'
wandb.init(
    project=os.environ["WANDB_PROJECT"],
    name=RUN_NAME,
    id=run_id,         # resume previous run if available
    resume="allow",    # allows resuming crashed run
)


RESUME_TRAINING = True
OUTPUT_DIR = "./lora-final_model_all_linear_r64-output"
PER_DEVICE_BATCH_SIZE = 2  # higher values --> OOM

optimizer = 'nadam'
effective_batch_size = 16
learning_rate = 1e-5
weight_decay = 0.0
betas = (0.9, 0.9999)
warmup_ratio = 0.2
epochs = 1
gradient_accumulation_steps = int(effective_batch_size / PER_DEVICE_BATCH_SIZE)
lora_r = 16*4
lora_alpha = 64*4
lora_dropout = 0.01

optimizer_map = {
    "adam": torch.optim.Adam,
    "adamw": torch.optim.AdamW,
    "nadam": torch.optim.NAdam,
    "adamax": torch.optim.Adamax
}

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    lr_scheduler_type="cosine",
    warmup_ratio=warmup_ratio,
    save_strategy="steps",
    save_steps=gradient_accumulation_steps*5,
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=gradient_accumulation_steps*5,
    logging_strategy="steps",
    logging_steps=gradient_accumulation_steps*5,
    report_to=['wandb'],
    run_name=RUN_NAME,
    fp16=True,
    fp16_full_eval=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    max_grad_norm=1,
    load_best_model_at_end=True,
    gradient_checkpointing=True
)

def build_optimizer(model):
    optimizer_class = optimizer_map[optimizer]
    return optimizer_class(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay,
        betas=betas
    )


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules='all-linear'
)
model.requires_grad_(False)                     # freeze base weights (precautionary)
model_peft = get_peft_model(model, peft_config) # inject a LoRA adapter

trainer = SFTTrainer(
    model=model_peft,
    train_dataset=ds_train_with_assistant_content,
    eval_dataset=ds_valid_with_assistant_content,
    formatting_func=formatting_func,
    args=training_args,
    optimizers=(build_optimizer(model_peft), None),  # (optimizer, scheduler)
    callbacks=[EarlyStoppingCallback(early_stopping_patience=25)]
)


# Training setup summary
dataset_size = len(ds_train_with_assistant_content)
steps_per_epoch = dataset_size // (PER_DEVICE_BATCH_SIZE * gradient_accumulation_steps)
total_steps = steps_per_epoch * epochs
warmup_steps = int(total_steps * warmup_ratio)

print("===== Training Setup Summary =====")
print(f"Num epochs:            {epochs}")
print(f"Effective batch size:  {effective_batch_size}")
print(f"Per-device batch size: {PER_DEVICE_BATCH_SIZE}")
print(f"Gradient accumulation: {gradient_accumulation_steps}")
print(f"Dataset size:          {dataset_size}")
print(f"Steps per epoch:       {steps_per_epoch}")
print(f"Total training steps:  {total_steps}")
print(f"Warmup steps:          {warmup_steps}")
print(f"Logging steps:         {training_args.logging_steps}")
print("===================================")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")


# Training
last_checkpoint = None
if RESUME_TRAINING and os.path.isdir(OUTPUT_DIR):
    last_checkpoint = get_last_checkpoint(OUTPUT_DIR)

if last_checkpoint is not None:
    print(f"Resuming training from checkpoint: {last_checkpoint}")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("Starting fresh training run")
    trainer.train()

print(f"End time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")


# WandB logging of eval metrics
for log in trainer.state.log_history:
    if 'eval_loss' in log:
        wandb.log({
            "eval_loss": log['eval_loss'],
            "eval_perplexity": math.exp(log['eval_loss']),
            "step": log['step'],
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "betas": betas,
            "warmup_ratio": warmup_ratio,
            "effective_batch_size": effective_batch_size,
            "optimizer": optimizer
        })

wandb.finish()  # finish the run

In [14]:
model_path = os.path.join(OUTPUT_DIR, 'final')
trainer.save_model(model_path)

In [ ]:
BATCH_SIZE = 32
ENABLE_THINKING = False
MAX_NEW_TOKENS = 512


prompts = [ds_test[id]['sql_prompt'] for id in range(len(ds_test))]
contexts = [ds_test[id]['sql_context'] for id in range(len(ds_test))]

responses = []
print(f"Start time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")
for i in tqdm(range(0, len(prompts), BATCH_SIZE)):
    batch_prompts = prompts[i : i + BATCH_SIZE]
    batch_contexts = contexts[i : i + BATCH_SIZE]

    messages_list = [
        construct_message(prompt=p, context=c)
        for p, c in zip(batch_prompts, batch_contexts)
    ]

    batch_responses = generate_model_response_batch(model, tokenizer, messages_list, enable_thinking=ENABLE_THINKING, max_new_tokens=MAX_NEW_TOKENS)

    responses.extend(batch_responses)

print(f"End time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")

## Test

In [17]:
torch.cuda.empty_cache()
get_vm_usage_metrics()

CPU 0 load: 0.00
CPU 1 load: 3.00
CPU 2 load: 0.00
CPU 3 load: 0.00
RAM Total: 27.41 GB, Used: 1.93 GB
GPU 0 (Tesla T4) load: 0.0%
GPU 0 (Tesla T4) VRAM Total: 16384.0 MB, Used 2407.0 MB
Disk Total: 60.95 GB, Used: 35.11 GB


In [22]:
model_path = './lora-final_model_all_linear_r64-output/final'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

print_trainable_parameters(model)
model.eval()

Trainable params: 0 || All params: 636420096 || Trainable %: 0.00


Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): lora.Linear(
            (base_layer): Linear(in_features=1024, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.01, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=1024, out_features=64, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=64, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default

In [14]:
BATCH_SIZE = 32
ENABLE_THINKING = False
MAX_NEW_TOKENS = 512


prompts = [ds_test[id]['sql_prompt'] for id in range(len(ds_test))]
contexts = [ds_test[id]['sql_context'] for id in range(len(ds_test))]

responses = []
print(f"Start time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")
for i in tqdm(range(0, len(prompts), BATCH_SIZE)):
    batch_prompts = prompts[i : i + BATCH_SIZE]
    batch_contexts = contexts[i : i + BATCH_SIZE]

    messages_list = [
        construct_message(prompt=p, context=c)
        for p, c in zip(batch_prompts, batch_contexts)
    ]

    batch_responses = generate_model_response_batch(model, tokenizer, messages_list, enable_thinking=ENABLE_THINKING, max_new_tokens=MAX_NEW_TOKENS)

    responses.extend(batch_responses)

print(f"End time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")

Start time: 2025-10-08_06-35-48


  0%|          | 0/183 [00:00<?, ?it/s]

End time: 2025-10-08_07-25-10


In [15]:
references = [ds_test[id]['sql'] for id in range(len(ds_test))]
predictions = [responses[id]['content'] for id in range(len(ds_test))]

scores = [
    evaluate_sql_response(
        reference=reference,
        prediction=prediction,
        sql_context=context
    )
    for reference, prediction, context in tqdm(zip(references, predictions, contexts), total=len(ds_test))
]

  0%|          | 0/5851 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [16]:
print(f"Mean test set score: {np.mean([score['final_score'] for score in scores]):.3f}")

Mean test set score: 0.774


# QLoRA

In [14]:
ds_train_with_assistant_content = ds_train.map(construct_message_with_assistant_content)
ds_valid_with_assistant_content = ds_valid.map(construct_message_with_assistant_content)

In [ ]:
import torch
import math
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback,
    TrainerCallback  # Added this import
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
import wandb
import os
from datetime import datetime

# ===== MODEL LOADING =====
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

checkpoint = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Set padding token if needed
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    checkpoint, 
    quantization_config=quantization_config,
    device_map='auto',
    trust_remote_code=True,
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# ===== TRAINING CONFIGURATION =====
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
RUN_NAME = f'qlora-fixed-{timestamp}'

wandb.init(
    project=os.environ["WANDB_PROJECT"],
    name=RUN_NAME,
    resume="allow",
)

RESUME_TRAINING = True
OUTPUT_DIR = "./qlora-fixed-output"
PER_DEVICE_BATCH_SIZE = 2

# Hyperparameters (adjusted for QLoRA)
optimizer = 'paged_adamw_8bit'
effective_batch_size = 16
learning_rate = 2e-4  # Lower for QLoRA
weight_decay = 0.01   # Added regularization
betas = (0.9, 0.999)
warmup_ratio = 0.1    # Reduced warmup
epochs = 1
gradient_accumulation_steps = int(effective_batch_size / PER_DEVICE_BATCH_SIZE)
lora_r = 64           # Reasonable rank
lora_alpha = 16       # alpha ~ r/4
lora_dropout = 0.1    # Slightly higher dropout

# ===== LoRA CONFIG =====
peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules='all-linear'  # Keep your working target modules
)

model_peft = get_peft_model(model, peft_config)

# Print trainable parameters
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )
    return trainable_params

print_trainable_parameters(model_peft)

# ===== GRADIENT DEBUGGING CALLBACK =====
class GradientDebugCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % args.logging_steps == 0 and state.global_step > 0:
            model = kwargs['model']
            total_grad_norm = 0
            trainable_params = 0
            for name, param in model.named_parameters():
                if param.requires_grad and param.grad is not None:
                    total_grad_norm += param.grad.norm().item()
                    trainable_params += 1
            
            if trainable_params > 0:
                avg_grad_norm = total_grad_norm / trainable_params
                print(f"Step {state.global_step}: Average gradient norm: {avg_grad_norm:.6f}")
                wandb.log({"avg_gradient_norm": avg_grad_norm, "step": state.global_step})

# ===== TRAINING ARGUMENTS (FIXED) =====
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    lr_scheduler_type="cosine",
    warmup_ratio=warmup_ratio,
    save_strategy="steps",
    save_steps=gradient_accumulation_steps * 5,
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=gradient_accumulation_steps * 5,
    logging_strategy="steps",
    logging_steps=gradient_accumulation_steps,
    report_to=['wandb'],
    run_name=RUN_NAME,
    
    # Critical QLoRA settings
    bf16=True,                    # Use bf16 instead of fp16
    fp16=False,                   # Disable fp16 when using bf16
    dataloader_pin_memory=False,
    
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    max_grad_norm=0.3,           # Reduced gradient clipping
    load_best_model_at_end=True,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    optim=optimizer,
    
    # Additional stability settings
    remove_unused_columns=False,
    ddp_find_unused_parameters=False,
)

# ===== TRAINER SETUP =====
trainer = SFTTrainer(
    model=model_peft,
    train_dataset=ds_train_with_assistant_content,
    eval_dataset=ds_valid_with_assistant_content,
    formatting_func=formatting_func,
    args=training_args,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=25),
        GradientDebugCallback()  # Add gradient monitoring
    ]
)

# ===== TRAINING SUMMARY =====
dataset_size = len(ds_train_with_assistant_content)
steps_per_epoch = dataset_size // (PER_DEVICE_BATCH_SIZE * gradient_accumulation_steps)
total_steps = steps_per_epoch * epochs
warmup_steps = int(total_steps * warmup_ratio)

print("===== TRAINING SETUP SUMMARY =====")
print(f"Model: {checkpoint}")
print(f"Num epochs: {epochs}")
print(f"Effective batch size: {effective_batch_size}")
print(f"Learning rate: {learning_rate}")
print(f"LoRA rank (r): {lora_r}")
print(f"LoRA alpha: {lora_alpha}")
print(f"Dataset size: {dataset_size}")
print(f"Total training steps: {total_steps}")
print(f"Warmup steps: {warmup_steps}")
print("===================================")

# ===== START TRAINING =====
print(f"Start time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")

last_checkpoint = None
if RESUME_TRAINING and os.path.isdir(OUTPUT_DIR):
    from transformers.trainer_utils import get_last_checkpoint
    last_checkpoint = get_last_checkpoint(OUTPUT_DIR)

if last_checkpoint is not None:
    print(f"Resuming training from checkpoint: {last_checkpoint}")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("Starting fresh training run")
    trainer.train()

print(f"End time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")

# ===== LOG FINAL RESULTS =====
for log in trainer.state.log_history:
    if 'eval_loss' in log:
        wandb.log({
            "eval_loss": log['eval_loss'],
            "eval_perplexity": math.exp(log['eval_loss']),
            "step": log['step'],
        })

# Save the final model
trainer.save_model()
print(f"Model saved to {OUTPUT_DIR}")

wandb.finish()

trainable params: 40370176 || all params: 416219136 || trainable%: 9.70


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


===== TRAINING SETUP SUMMARY =====
Model: Qwen/Qwen3-0.6B
Num epochs: 1
Effective batch size: 16
Learning rate: 0.0002
LoRA rank (r): 64
LoRA alpha: 16
Dataset size: 97500
Total training steps: 6093
Warmup steps: 609
Start time: 2025-10-21_14-16-20
Starting fresh training run


Step,Training Loss,Validation Loss


In [32]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

checkpoint = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint, 
    quantization_config=quantization_config,
    device_map='auto',
    trust_remote_code=True,  # Important for Qwen
)
# model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
# print_trainable_parameters(model)
# model.eval()

In [31]:
import torch
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Reset and try a fresh approach
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

checkpoint = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Load model with proper settings
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,  # Important for Qwen
)

# Set tokenizer if needed
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Critical: Prepare model BEFORE applying LoRA
model = prepare_model_for_kbit_training(model)

# Use a conservative LoRA config
# peft_config = LoraConfig(
#     r=16,  # Lower rank for stability
#     lora_alpha=32,
#     lora_dropout=0.1,
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules='all-linear'
# )

peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules='all-linear'
)
model_peft = get_peft_model(model, peft_config)

# Verify trainable parameters
trainable_params = 0
all_params = 0
for name, param in model_peft.named_parameters():
    all_params += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
        # print(f"Trainable: {name}")

print(f"Trainable parameters: {trainable_params:_} / {all_params:_}")
print(f"Percentage: {100 * trainable_params / all_params:.4f}%")

Trainable parameters: 10_092_544 / 385_941_504
Percentage: 2.6150%


In [35]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

checkpoint = "Qwen/Qwen3-0.6B"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint, 
    quantization_config=quantization_config,
    device_map='auto',
    trust_remote_code=True,  # Important for Qwen
)
# model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
# print_trainable_parameters(model)
# model.eval()


peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules='all-linear'
)

model_peft = get_peft_model(model, peft_config)

In [36]:
from torch.utils.data import DataLoader

# After model_peft is created
print("=== GRADIENT DEBUGGING ===")

# Test forward/backward pass - handle dataset format properly
try:
    # Try accessing the first sample to understand the structure
    sample = ds_train_with_assistant_content[0]
    print(f"Sample structure: {type(sample)}")
    print(f"Sample keys: {sample.keys() if hasattr(sample, 'keys') else 'No keys'}")
    print(f"Sample content: {sample}")
    
    # Format the first 2 samples
    formatted_texts = []
    for i in range(2):
        sample = ds_train_with_assistant_content[i]
        formatted = formatting_func(sample)
        formatted_texts.append(formatted)
        print(f"Sample {i} formatted length: {len(formatted)}")
    
    # Tokenize
    inputs = tokenizer(formatted_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(model_peft.device) for k, v in inputs.items()}
    inputs["labels"] = inputs["input_ids"].clone()

    print(f"Input shape: {inputs['input_ids'].shape}")

    # Forward pass
    model_peft.train()  # Ensure training mode
    outputs = model_peft(**inputs)
    loss = outputs.loss
    print(f"Initial loss: {loss.item()}")

    # Zero gradients first
    model_peft.zero_grad()

    # Backward pass
    loss.backward()

    # Check gradients
    total_grad_norm = 0
    trainable_params_with_grad = 0
    for name, param in model_peft.named_parameters():
        if param.requires_grad and param.grad is not None:
            grad_norm = param.grad.norm().item()
            total_grad_norm += grad_norm
            trainable_params_with_grad += 1
            if trainable_params_with_grad <= 5:  # Print first 5
                print(f"Gradient found for: {name} - norm: {grad_norm}")

    print(f"Total trainable parameters with gradients: {trainable_params_with_grad}")
    print(f"Total gradient norm: {total_grad_norm}")

    if trainable_params_with_grad == 0:
        print("❌ CRITICAL: No gradients detected! The model cannot learn.")
    else:
        print("✅ Gradients are flowing")

except Exception as e:
    print(f"Error during debugging: {e}")
    print("Let's try a different approach...")
    
    # Alternative: Create simple test data
    print("Using simple test data...")
    test_texts = ["Hello, how are you?", "This is a test sentence."]
    inputs = tokenizer(test_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(model_peft.device) for k, v in inputs.items()}
    inputs["labels"] = inputs["input_ids"].clone()
    
    # Forward pass
    model_peft.train()
    outputs = model_peft(**inputs)
    loss = outputs.loss
    print(f"Test loss with simple data: {loss.item()}")
    
    # Backward pass
    model_peft.zero_grad()
    loss.backward()
    
    # Check gradients
    trainable_params_with_grad = 0
    for name, param in model_peft.named_parameters():
        if param.requires_grad and param.grad is not None:
            trainable_params_with_grad += 1
            if trainable_params_with_grad <= 3:
                print(f"Gradient found for: {name}")
    
    print(f"Total parameters with gradients: {trainable_params_with_grad}")

=== GRADIENT DEBUGGING ===
Sample structure: <class 'dict'>
Sample keys: dict_keys(['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation', 'messages'])
Sample content: {'id': 29981, 'domain': 'precision agriculture', 'domain_description': 'Precision farming data, satellite imagery analysis, IoT sensor metrics, and agricultural automation trends.', 'sql_complexity': 'aggregation', 'sql_complexity_description': 'aggregation functions (COUNT, SUM, AVG, MIN, MAX, etc.), and HAVING clause', 'sql_task_type': 'analytics and reporting', 'sql_task_type_description': 'generating reports, dashboards, and analytical insights', 'sql_prompt': 'What is the average moisture level for each crop type in the past month?', 'sql_context': 'CREATE TABLE crop_moisture (id INT, crop_id INT, type VARCHAR(255), moisture FLOAT, timestamp DATETIME);', 'sql': 'SELECT type, AVG(moistur

In [13]:
# torch.cuda.empty_cache()
get_vm_usage_metrics()

print(len(ds_train), len(ds_valid), len(ds_test))

ds_train_with_assistant_content = ds_train.map(construct_message_with_assistant_content)
ds_valid_with_assistant_content = ds_valid.map(construct_message_with_assistant_content)

CPU 0 load: 0.00
CPU 1 load: 0.00
CPU 2 load: 2.00
CPU 3 load: 0.00
RAM Total: 27.41 GB, Used: 2.28 GB
GPU 0 (Tesla T4) load: 0.0%
GPU 0 (Tesla T4) VRAM Total: 16384.0 MB, Used 1713.0 MB
Disk Total: 60.95 GB, Used: 37.41 GB
97500 2500 5851


In [14]:
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
RUN_NAME = f'qlora-final-model-all-linear-r64-{timestamp}'

wandb.init(
    project=os.environ["WANDB_PROJECT"],
    name=RUN_NAME,
    resume="allow",
)

RESUME_TRAINING = True
OUTPUT_DIR = "./qlora-final_model_all_linear_r64-output"
PER_DEVICE_BATCH_SIZE = 2


optimizer = 'paged_adamw_8bit'  # better for 4-bit models
effective_batch_size = 16
learning_rate = 1e-3#5
weight_decay = 0.0
betas = (0.9, 0.999)
warmup_ratio = 0.2
epochs = 1
gradient_accumulation_steps = int(effective_batch_size / PER_DEVICE_BATCH_SIZE)
lora_r = 16*4
lora_alpha = 64*4
lora_dropout = 0.01


training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    num_train_epochs=epochs,
    weight_decay=weight_decay,
    lr_scheduler_type="cosine",
    warmup_ratio=warmup_ratio,
    save_strategy="steps",
    save_steps=gradient_accumulation_steps*5,
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=gradient_accumulation_steps*5,
    logging_strategy="steps",
    logging_steps=gradient_accumulation_steps*5,
    report_to=['wandb'],
    run_name=RUN_NAME,
    
    # fp16=True,
    bf16=True,
    # fp16_full_eval=False,
    
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    max_grad_norm=1.0,
    load_best_model_at_end=True,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},
    optim=optimizer, # built-in optimizer
)

peft_config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
    # target_modules='all-linear'
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
        "lm_head", "wte"  # Add embedding layer
    ]
)
peft_config.inference_mode = False

model_peft = get_peft_model(model, peft_config)
print_trainable_parameters(model_peft)

trainer = SFTTrainer(
    model=model_peft,
    train_dataset=ds_train_with_assistant_content,
    eval_dataset=ds_valid_with_assistant_content,
    formatting_func=formatting_func,
    args=training_args,
    # optimizers=(build_optimizer(model_peft), None),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=25)]
)

dataset_size = len(ds_train_with_assistant_content)
steps_per_epoch = dataset_size // (PER_DEVICE_BATCH_SIZE * gradient_accumulation_steps)
total_steps = steps_per_epoch * epochs
warmup_steps = int(total_steps * warmup_ratio)

print("===== Training Setup Summary =====")
print(f"Num epochs:            {epochs}")
print(f"Effective batch size:  {effective_batch_size}")
print(f"Per-device batch size: {PER_DEVICE_BATCH_SIZE}")
print(f"Gradient accumulation: {gradient_accumulation_steps}")
print(f"Dataset size:          {dataset_size}")
print(f"Steps per epoch:       {steps_per_epoch}")
print(f"Total training steps:  {total_steps}")
print(f"Warmup steps:          {warmup_steps}")
print(f"Logging steps:         {training_args.logging_steps}")
print("===================================")
print(f"Start time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")

last_checkpoint = None
if RESUME_TRAINING and os.path.isdir(OUTPUT_DIR):
    last_checkpoint = get_last_checkpoint(OUTPUT_DIR)

if last_checkpoint is not None:
    print(f"Resuming training from checkpoint: {last_checkpoint}")
    trainer.train(resume_from_checkpoint=last_checkpoint)
else:
    print("Starting fresh training run")
    trainer.train()

print(f"End time: {datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}")


for log in trainer.state.log_history:
    if 'eval_loss' in log:
        wandb.log({
            "eval_loss": log['eval_loss'],
            "eval_perplexity": math.exp(log['eval_loss']),
            "step": log['step'],
            "learning_rate": learning_rate,
            "weight_decay": weight_decay,
            "betas": betas,
            "warmup_ratio": warmup_ratio,
            "effective_batch_size": effective_batch_size,
            "optimizer": optimizer
        })

wandb.finish()

wandb: Currently logged in as: olialeshka (olialeshka-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/olialeshka_1/myenv/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:693: UserWarning: Model with `tie_word_embeddings=True` and the tied_target_modules=['lm_head'] are part of the adapter. This can lead to complications, for example when merging the adapter or converting your model to formats other than safetensors. See for example https://github.com/huggingface/peft/issues/2018.
  warnings.warn(


Trainable params: 50159616 || All params: 426008576 || Trainable %: 11.77


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


===== Training Setup Summary =====
Num epochs:            1
Effective batch size:  16
Per-device batch size: 2
Gradient accumulation: 8
Dataset size:          97500
Steps per epoch:       6093
Total training steps:  6093
Warmup steps:          1218
Logging steps:         40
Start time: 2025-10-21_13-11-32
Starting fresh training run


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
40,2.361800,2.382753,1.283656,124369.000000,0.648450
80,2.379700,2.382753,1.283656,246617.000000,0.648450


/home/olialeshka_1/myenv/lib/python3.12/site-packages/peft/utils/save_and_load.py:270: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


KeyboardInterrupt: 